# Hiperparámetros de árboles de decisión: **de la teoría a su implementación en scikit-learn**

Los árboles de decisión controlan su complejidad mediante un conjunto de hiperparámetros que regulan **qué tan fino** es el particionado del espacio de características y **cuándo** dejar de crecer o **cómo** podar. A continuación se presentan estos controles **en términos teóricos (siguiendo la notación de *Introduction to Statistical Learning*)**, y luego su **mapeo directo** a `DecisionTreeClassifier` de *scikit-learn*, con notas prácticas sobre su efecto en **sesgo–varianza**, estabilidad y eficiencia.

---

## 1) Criterio de partición (función de impureza / ganancia de información)

**Teoría.**  
Sea $p_{mk}$ la proporción de observaciones en el nodo $m$ que pertenecen a la clase $k$ (con $k=1,\dots,K$). La impureza mide cuán “mezclado” está un nodo.

- **Índice de Gini**:
$$
G(m) = \sum_{k=1}^K p_{mk}(1 - p_{mk}) \;=\; 1 - \sum_{k=1}^K p_{mk}^2
$$

- **Entropía**:
$$
H(m) = - \sum_{k=1}^K p_{mk}\,\log(p_{mk})
$$

La reducción en impureza al dividir un nodo $m$ en hijos izquierdo ($L$) y derecho ($R$) se calcula como:
$$
\Delta = \mathrm{Impureza}(m) \;-\; \frac{N_L}{N_m}\,\mathrm{Impureza}(L) \;-\; \frac{N_R}{N_m}\,\mathrm{Impureza}(R),
$$
donde $N_m$ es el número de observaciones en $m$, y $N_L$, $N_R$ corresponden a los nodos hijos.

**En scikit-learn.** `criterion={"gini","entropy","log_loss"}`

---

## 2) Estrategia de búsqueda del split

**Teoría.** Al evaluar divisiones, se puede elegir de forma determinista la mejor según el criterio, o introducir aleatoriedad para generar árboles distintos (útil en ensambles).

**En scikit-learn.** `splitter={"best","random"}`  
- `best`: busca exhaustivamente el mejor split.  
- `random`: selecciona aleatoriamente un subconjunto de splits candidatos.

---

## 3) Criterios de **pre-poda** (detener el crecimiento antes de tiempo)

**Teoría.** El crecimiento de un árbol sin restricciones tiende a generar sobreajuste: baja varianza en entrenamiento pero mala generalización. Para evitarlo, se fijan umbrales de complejidad:

- **Profundidad máxima ($\max\_depth$).** Limita el número de niveles. Árboles más profundos capturan más detalle, pero arriesgan sobreajuste.  
- **Número mínimo de observaciones para dividir ($\min\_samples\_split$).** Un nodo sólo se divide si contiene al menos ese número de observaciones.  
- **Número mínimo de observaciones por hoja ($\min\_samples\_leaf$).** Obliga a que cada hoja contenga una fracción significativa de observaciones.  
- **Mínima reducción de impureza ($\min\_impurity\_decrease$).** Un split sólo se realiza si reduce la impureza al menos en esa cantidad.  
- **Número máximo de hojas ($\max\_leaf\_nodes$).** Limita directamente la cantidad de nodos terminales.  
- **Fracción mínima de peso en hojas ($\min\_weight\_fraction\_leaf$).** Variante de `min_samples_leaf` cuando se usan pesos.

**En scikit-learn.**
```python
DecisionTreeClassifier(
  max_depth=6,
  min_samples_split=10,
  min_samples_leaf=5,
  min_impurity_decrease=1e-3,
  max_leaf_nodes=None,
  min_weight_fraction_leaf=0.0,
  random_state=42
)
```
Estos parámetros permiten reducir la varianza y mejoran la generalización.

---

## 4) Submuestreo de variables en cada split

**Teoría.** En cada nodo puede evaluarse un subconjunto de variables en lugar de todas. Esto introduce variabilidad y ayuda a reducir la dependencia de una única característica dominante. En ensambles como Random Forests, este mecanismo es esencial para reducir la correlación entre árboles.

**En scikit-learn.**  
Parámetro: `max_features: int | float | {"sqrt","log2"} | None`  

- `None`: se usan todas las variables.  
- `sqrt`: usa la raíz cuadrada del número total de variables.  
- `log2`: usa el logaritmo base 2 del número total de variables.  
- `int` o `float`: define cuántas variables (o fracción) se consideran en cada split.

**Efecto.**  
- Con todas las variables, el árbol es más flexible, pero puede sobreajustar.  
- Al limitar `max_features`, se introduce más sesgo pero menos varianza.  
- Es muy útil cuando $p \gg n$ (muchas variables respecto al número de observaciones) o cuando existen variables irrelevantes.

---

## 5) **Post-poda** por costo-complejidad

**Teoría.** Una vez crecido el árbol, se aplica la poda basada en costo-complejidad:
$$
R_\alpha(T) = R(T) + \alpha\,|T|,
$$
donde $R(T)$ es el error de entrenamiento del árbol $T$, $|T|$ es el número de hojas, y $\alpha \geq 0$ es un parámetro de penalización. A mayor $\alpha$, más se favorecen árboles pequeños.

**En scikit-learn.**  
Parámetro: `ccp_alpha: float ≥ 0`  

- `0.0`: sin poda.  
- Valores positivos: se eliminan ramas con poca contribución.

**Práctica recomendada.**  
Utilizar el método `cost_complexity_pruning_path` para explorar el camino de poda y seleccionar $\alpha$ mediante validación cruzada.
(ver https://scikit-learn.org/stable/auto_examples/tree/plot_cost_complexity_pruning.html)

**Efecto práctico.**  
- Disminuye la varianza y mejora interpretabilidad.  
- La elección adecuada de $\alpha$ permite mantener un buen desempeño en test, evitando árboles demasiado complejos.

---

## 6) Ponderación y clases desbalanceadas

**Teoría.** En problemas desbalanceados, los errores de la clase minoritaria deben recibir más peso. Esto evita que el árbol favorezca desproporcionadamente a la clase mayoritaria.

**En scikit-learn.**  
- `class_weight={"balanced", dict}`: ajusta automáticamente los pesos en función de la frecuencia de clases o permite definirlos manualmente.  
- `sample_weight` en `fit(...)`: pondera instancias específicas.

**Efecto.**  
- Cambia los cálculos de impureza y, por ende, los umbrales óptimos de split.  
- Mejora el recall de clases minoritarias, aunque puede reducir la precisión en la clase mayoritaria.

---

## 7) Naturaleza de las variables y datos faltantes

**Teoría.**  
- Variables continuas: los árboles las dividen mediante umbrales.  
- Variables categóricas: en teoría, un árbol puede dividir según subconjuntos de categorías.  
- Datos faltantes: algunos algoritmos incorporan *surrogate splits* para manejar ausencias.

**En scikit-learn.**  
- No soporta categóricas puras de forma nativa → se requiere codificación previa (one-hot, ordinal, etc.).  
- No maneja `NaN` directamente → se recomienda imputación (ej. `SimpleImputer`).

**Buenas prácticas.**  
Encadenar preprocesamiento y modelo en un `Pipeline` para evitar *data leakage* durante validación cruzada.

---

## 8) Aleatoriedad y reproducibilidad

**Teoría.** Si existen empates en los splits, el algoritmo puede dar árboles distintos en ejecuciones diferentes. Para garantizar resultados reproducibles, se fija una semilla de aleatoriedad.

**En scikit-learn.**  
Parámetro: `random_state`

**Efecto.**  
- Permite reproducibilidad en experimentos.  
- Fundamental al comparar configuraciones o al enseñar.

---

## 9) Métrica de evaluación y protocolo de validación

**Teoría.** El ajuste de hiperparámetros depende de la métrica y protocolo usados para evaluar el modelo. Accuracy es común, pero en problemas desbalanceados conviene F1 o ROC-AUC. La validación cruzada estratificada es preferida para árboles de clasificación.

**En scikit-learn.**  
- Parámetro `scoring` en GridSearchCV/RandomizedSearchCV (`"accuracy"`, `"f1"`, `"roc_auc"`, etc.).  
- Esquemas de CV como `StratifiedKFold`.

**Efecto.**  
La elección de métrica determina qué configuraciones se consideran “óptimas”. En datasets desbalanceados, usar sólo accuracy puede inducir a modelos engañosamente buenos.

---

## 10) Itinerarios de *tuning* y recomendaciones

1. **Comenzar con pre-poda simple**: fijar `max_depth` y `min_samples_leaf` para evitar sobreajuste evidente.  
2. **Refinar** con `min_samples_split` y, si hay muchas variables, con `max_features`.  
3. **Aplicar post-poda** (`ccp_alpha`) validada por CV para simplificar el modelo sin sacrificar rendimiento.  
4. **Seleccionar la métrica** adecuada al problema (ej. F1 o ROC-AUC en desbalance).  
5. **Validar cuidadosamente** con esquemas estratificados y, cuando sea necesario, CV anidada.  
6. Para espacios grandes de búsqueda, preferir **Optuna** u optimizadores bayesianos en lugar de GridSearch.

---

## Resumen

Los hiperparámetros de los árboles de decisión corresponden a mecanismos de **control de complejidad** (pre-poda, submuestreo) y de **regularización explícita** (post-poda). Su elección debe ser **consciente**, en función del balance sesgo–varianza y de la métrica de interés. Ajustarlos correctamente permite obtener modelos que generalicen bien y que sean interpretables.

